<a href="https://colab.research.google.com/github/alik604/Notebooks/blob/master/amonalyDetection/kddcup_99_100accAchieved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection 

### Dataset from https://www.kaggle.com/bharath5673/kddcup99

### Sample code used: https://www.kaggle.com/nidhirastogi/intrusion-detection/data


In [0]:
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np
import re
import sklearn

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib as matplot
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.model_selection import train_test_split

In [7]:
import os

os.environ['KAGGLE_USERNAME'] = "alik604" # username from the json file os.environ['KAGGLEKEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx" # key from the json file
os.environ['KAGGLE_KEY'] = "1c6c5de0f6a4d1610a2f8f5ad6771bb7"

!mkdir /root/.kaggle
!echo '{"username":"alik604","key":"1c6c5de0f6a4d1610a2f8f5ad6771bb7"}' > /root/.kaggle/kaggle.json

!kaggle datasets download -d bharath5673/kddcup99
!ls

mkdir: cannot create directory ‘/root/.kaggle’: File exists
kddcup99.zip: Skipping, found more recently modified local copy (use --force to force download)
kddcup99.zip  kddcup.data  kddcup.data.corrected  sample_data


In [8]:
import zipfile

zip_ref = zipfile.ZipFile('kddcup99.zip', 'r')
zip_ref.extractall()
zip_ref.close()
!ls

kddcup99.zip  kddcup.data  kddcup.data.corrected  sample_data


In [9]:
!cd kddcup.data/
!ls

kddcup99.zip  kddcup.data  kddcup.data.corrected  sample_data


In [19]:
train = pd.read_csv('./kddcup.data')
train.head(5)

,0,tcp,http,SF,215,45076,0.1,0.2,0.3,0.4,0.5,1,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,1.1,1.2,0.00,0.00.1,0.00.2,0.00.3,1.00,0.00.4,0.00.5,0.16,0.17,0.00.6,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,0.00.12,0.00.13,normal.
0,0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


### Reduce train size for faster trainin, remove when in production

In [20]:
train.columns =["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
train.shape
train.head(5)

(4898430, 42)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


### The following few cells are taken from the 'sample code'

In [55]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


#set(list(train['label']))
train['protocol_type'] = le.fit_transform(train['protocol_type'])
train['service'] = le.fit_transform(train['service'])
train['flag'] = le.fit_transform(train['flag'])

train['label'] = le.fit_transform(train['label'])

train['label'].describe()

print('\n')
tmp = train.corr().abs().sort_values('label')
garbage = tmp['label'].nsmallest(10)

garbage = list(garbage.index)
garbage


count    4.898430e+06
mean     1.458601e+01
std      4.054754e+00
min      0.000000e+00
25%      1.100000e+01
50%      1.800000e+01
75%      1.800000e+01
max      2.200000e+01
Name: label, dtype: float64

['src_bytes',
 'is_host_login',
 'dst_bytes',
 'urgent',
 'num_compromised',
 'num_root',
 'su_attempted',
 'land',
 'num_failed_logins',
 'num_shells']

In [0]:
data_x = train.drop('label', axis=1)
data_y = train.loc[:,['label']]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.95, random_state=42) # TODO

## Drop features with lowest STD

In [72]:

#drop n smallest std features
tmp =  X_train.std().to_frame().nsmallest(10, columns=0)
tmp = list(tmp.transpose().columns)  #fuckin pandas.core.indexes.base.Index   -_-
#tmp = tmp.append('num_outbound_cmds')  # might not work...
tmp
tmp = set(tmp + garbage)
len(tmp)
tmp

['num_outbound_cmds',
 'is_host_login',
 'land',
 'urgent',
 'root_shell',
 'su_attempted',
 'num_shells',
 'num_failed_logins',
 'is_guest_login',
 'num_access_files']

14

{'dst_bytes',
 'is_guest_login',
 'is_host_login',
 'land',
 'num_access_files',
 'num_compromised',
 'num_failed_logins',
 'num_outbound_cmds',
 'num_root',
 'num_shells',
 'root_shell',
 'src_bytes',
 'su_attempted',
 'urgent'}

In [54]:
print("Thats how to rid rid of {0} dimentions of data, from the 10 lowest STD and 10 lowest correlation".format(len(tmp)))

Thats how to rid rid of 11 dimentions of data, from the 10 lowest STD and 10 lowest correlation


In [73]:
from sklearn import linear_model

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

import gc 
gc.collect()

130

In [74]:
LR = linear_model.LinearRegression()
LR.fit(X_train, y_train)
lr_score = LR.score(X_test, y_test)
print('Linear regression processing')
print('Linear regression Score: %.2f %%' % lr_score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Linear regression processing
Linear regression Score: 0.92 %


In [78]:
try:  #TODO
    X_train = X_train.drop(tmp,axis=1)
    X_test = X_test.drop(tmp,axis=1)
except:
    None
    
X_train.shape
X_test.shape

(489843, 27)

(4408587, 27)

In [79]:
LR = linear_model.LinearRegression()
LR.fit(X_train, y_train)
lr_score = LR.score(X_test, y_test)
print('Linear regression processing ,,,')
print('Linear regression Score: %.2f %%' % lr_score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Linear regression processing ,,,
Linear regression Score: 0.98 %


In [0]:
AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, learning_rate=1.0)
RF = RandomForestClassifier(n_estimators=10, criterion='entropy', max_features='auto', bootstrap=True)
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')

In [0]:
# y_train = Y_train['xAttack'].ravel()
# x_train = X_train.values
# x_test = X_test.values

In [81]:
AB.fit(X_train, y_train)
AB_feature = AB.feature_importances_
#AB_feature
ab_score = AB.score(X_test, y_test)

print('AdaBoostClassifier processing ,,,')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                          

AdaBoostClassifier processing ,,,
AdaBoostClassifier Score: 1.000 %


In [82]:
RF.fit(X_train, y_train)
RF_feature = RF.feature_importances_
#RF_feature

rf_score = RF.score(X_test, y_test)

print('RandomForestClassifier processing ,,,')
print('RandomForestClassifier Score: %.3f %%' % rf_score)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

RandomForestClassifier processing ,,,
RandomForestClassifier Score: 1.000 %


In [83]:
ET.fit(X_train, y_train)
ET_feature = ET.feature_importances_
#ET_feature

et_score = ET.score(X_test, y_test)

print('ExtraTreesClassifier processing ,,,')
print('ExtraTreeClassifier: %.3f %%' % et_score)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

ExtraTreesClassifier processing ,,,
ExtraTreeClassifier: 1.000 %


In [84]:
GB.fit(X_train, y_train)

GB_feature = GB.feature_importances_
#GB_feature

gb_score = GB.score(X_test, y_test)

print('GradientBoostingClassifier processing ,,,')
print('GradientBoostingClassifier Score: %.3f %%' % gb_score)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features='auto', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

GradientBoostingClassifier processing ,,,
GradientBoostingClassifier Score: 0.996 %


In [85]:
feature_df = pd.DataFrame({'features': X_train.columns.values, # names
                           'AdaBoost' : AB_feature,
                           'RandomForest' : RF_feature,
                           'ExtraTree' : ET_feature,
                           'GradientBoost' : GB_feature
                          })
#feature_df.features
feature_df.head(2)

,features,AdaBoost,RandomForest,ExtraTree,GradientBoost
0,duration,0.002133,0.000386,0.000184,0.003297
1,protocol_type,NaN,0.154119,0.162186,0.024564


In [86]:
n = 10
a_f = feature_df.nlargest(n, 'AdaBoost')
e_f = feature_df.nlargest(n, 'ExtraTree')
g_f = feature_df.nlargest(n, 'GradientBoost')
r_f = feature_df.nlargest(n, 'RandomForest')

result = pd.concat([a_f, e_f, g_f, r_f])
result = result.drop_duplicates() 
# result.shape
# print('\n')

# garbage = np.argsort(result.transpose().mean())
# garbage = garbage.sort_index()[:5] # FML... :'(
    
# garbage.index

# result = result.drop(garbage.index)
result



# arr = X_train.columns.to_numpy()
# result = result.set_index(np.take(arr,result.index))
result.index

Int64Index([13, 6, 5, 7, 0, 26, 4, 3, 15, 16, 9, 1, 21, 14, 8, 19, 11, 18, 22,
            20, 23, 2],
           dtype='int64')

### what the fuck am i doing with my life.... 45mins wasted on numpy 

In [88]:
# X_train_SF = X_train[result.index]
# X_test_SF = X_test[result.index]



selected_features = result['features'].values.tolist()
X_train_SF = X_train[selected_features]
X_test_SF = X_test[selected_features]


x = X_train_SF#.reshape(-1, 26)  # 31
y = y_train['label'].ravel()


# x=x[:20000]
# y=y[:20000]


x.shape
y.size

(489843, 22)

489843

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf1 = DecisionTreeClassifier() 
clf2 = RandomForestClassifier(n_estimators=25, random_state=1)# .77
clf3 = GradientBoostingClassifier() # .76
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 # without this lil fucker, Acc: 0.75 [Ensemble], 0.78 with 

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='hard') 
# n =7 with better selection; .79
# n =7 ; .77
# n =10 ; .78
# n =14 ; .77


# eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('et',ET)], voting='soft') #.76

for clf, label in zip([clf1, clf2, clf3,ET, eclf], ['DecisionTreeClassifier', 'Random Forest', 'GradientBoostingClassifier','ExtraTreesClassifier', 'Ensemble']): 
    # scores = cross_val_score(clf, x, y, cv=2, scoring='accuracy') # cv= 5 
    # print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

    tmp = clf.fit(x,y)
    pred = clf.score(X_test_SF,y_test)
    print("Acc: %0.2f [%s]" % (pred,label))

Acc: 1.00 [DecisionTreeClassifier]
Acc: 1.00 [Random Forest]
Acc: 1.00 [GradientBoostingClassifier]
Acc: 1.00 [ExtraTreesClassifier]
